In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
import sys
import os

warnings.filterwarnings('ignore')

# GLOBAL CONSTANTS AND DATA LOADING
FACTOR_GROUPS = {
    'Economy': ['GDP', 'Gini Index', 'Total natural resources rents (% of GDP)'],
    'Technology': ['R&D Expenditure (% of GDP)', 'Number of Patents Filed', 'Scientific Publications', 'Education Index'],
    'Space Capability': ['Space Agency Budget (US$ millions)', 'Most number of Objects Launched Into Space in a Year', 'Years of Space Program Experience'],
    'Society': ['Environmental Performance Index', 'Life Expectancy', 'Political Stability Index']
}
INPUT_INDICATORS = ['GDP', 'Gini Index', 'Total natural resources rents (% of GDP)', 'R&D Expenditure (% of GDP)', 'Education Index',
                    'Space Agency Budget (US$ millions)', 'Years of Space Program Experience']
OUTPUT_INDICATORS = ['Number of Patents Filed', 'Scientific Publications', 'Most number of Objects Launched Into Space in a Year',
                     'Environmental Performance Index', 'Life Expectancy', 'Political Stability Index']
INDICATOR_TYPES = {'Gini Index': 'cost', 'Total natural resources rents (% of GDP)': 'cost'}
for ind in (INPUT_INDICATORS + OUTPUT_INDICATORS):
    if ind not in INDICATOR_TYPES: INDICATOR_TYPES[ind] = 'benefit'
ASTEROID_CLASSES = {'I': 37.53, 'II': 32.81, 'III': 22.68}
SIMULATION_YEARS = range(2025, 2086)

def load_and_clean_data(filepath):
    try:
        df = pd.read_csv(filepath)
    except FileNotFoundError:
        print(f"Error: '{filepath}' not found.")
        return None
    national_df = df[df['Type'] == 'National'].copy()
    for col in national_df.select_dtypes(include=np.number).columns:
        if national_df[col].isnull().any():
            national_df[col].fillna(national_df[col].mean(), inplace=True)
    df.update(national_df)
    private_df = df[df['Type'] == 'Private'].copy()
    for idx, row in private_df.iterrows():
        home_country_data = df[(df['Country'] == row['Country']) & (df['Type'] == 'National')]
        if not home_country_data.empty:
            home_values = home_country_data.iloc[0]
            for col in df.columns:
                if pd.isna(df.at[idx, col]):
                    df.at[idx, col] = home_values.get(col, np.nan)
    print("Data loaded and cleaned successfully.")
    return df

# CORE METHODOLOGY FUNCTIONS
def calculate_ahp_weights(matrix, index):
    RI_TABLE = {1: 0, 2: 0, 3: 0.58, 4: 0.90}
    n = matrix.shape[0]
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    lambda_max = np.max(eigenvalues).real
    eigenvector = eigenvectors[:, eigenvalues.argmax()].real
    weights = eigenvector / eigenvector.sum()
    return pd.Series(weights, index=index)

def calculate_ewm_weights(data, indicators, indicator_types):
    df_ewm = data[indicators].copy()
    for col in indicators:
        if indicator_types.get(col) == 'cost':
            df_ewm[col] = df_ewm[col].max() - df_ewm[col]
    df_ewm = (df_ewm - df_ewm.min()) / (df_ewm.max() - df_ewm.min() + 1e-10)
    P = df_ewm / (df_ewm.sum(axis=0) + 1e-10)
    E = -1/np.log(len(df_ewm)) * (P * np.log(P + 1e-10)).sum(axis=0)
    d = 1 - E
    return pd.Series(d / (d.sum() + 1e-10), index=indicators)

def apply_topsis(data, indicators, weights, indicator_types):
    df_topsis = data[indicators].values
    aligned_weights = weights.reindex(indicators).values
    norm_data = df_topsis / (np.sqrt((df_topsis**2).sum(axis=0)) + 1e-10)
    weighted_data = norm_data * aligned_weights
    pis, nis = np.zeros(weighted_data.shape[1]), np.zeros(weighted_data.shape[1])
    for j, col in enumerate(indicators):
        if indicator_types.get(col) == 'benefit':
            pis[j], nis[j] = weighted_data[:, j].max(), weighted_data[:, j].min()
        else:
            pis[j], nis[j] = weighted_data[:, j].min(), weighted_data[:, j].max()
    dist_pis = np.sqrt(((weighted_data - pis)**2).sum(axis=1))
    dist_nis = np.sqrt(((weighted_data - nis)**2).sum(axis=1))
    return dist_nis / (dist_pis + dist_nis + 1e-10)

def calculate_amci(df):
    scaler = StandardScaler()
    df_amci_scaled = scaler.fit_transform(df[FACTOR_GROUPS['Space Capability']])
    pca = PCA(n_components=1)
    df['AMCI'] = pca.fit_transform(df_amci_scaled)[:, 0]
    df['AMCI'] = (df['AMCI'] - df['AMCI'].min()) / (df['AMCI'].max() - df['AMCI'].min()) * 100
    return df

# MAIN CALCULATION AND SIMULATION LOGIC
def calculate_eei(df, factor_groups, indicator_types, ahp_matrix):
    factor_ahp_weights = calculate_ahp_weights(ahp_matrix, index=factor_groups.keys())
    comprehensive_weights = {}
    for factor, indicators in factor_groups.items():
        all_factor_inds = [ind for ind in indicators if ind in INPUT_INDICATORS or ind in OUTPUT_INDICATORS]
        if not all_factor_inds: continue
        ewm_weights = calculate_ewm_weights(df, all_factor_inds, indicator_types)
        for indicator, ewm_weight in ewm_weights.items():
            comprehensive_weights[indicator] = factor_ahp_weights[factor] * ewm_weight
    comprehensive_weights_s = pd.Series(comprehensive_weights)
    input_scores = apply_topsis(df, INPUT_INDICATORS, comprehensive_weights_s, indicator_types)
    output_scores = apply_topsis(df, OUTPUT_INDICATORS, comprehensive_weights_s, indicator_types)
    df['Input_Score'] = input_scores * 10
    df['Output_Score'] = output_scores * 10
    df['EEI'] = np.abs(df['Input_Score'] - df['Output_Score'])
    return df, comprehensive_weights_s, factor_ahp_weights

def run_simulation(df_all_entities, initial_national_df, policy_name):
    current_national_df = initial_national_df.copy()
    history = []
    initial_amci = df_all_entities['AMCI'].copy()
    ahp_matrix_sim = np.array([[1, 2, 3, 4], [1/2, 1, 2, 3], [1/3, 1/2, 1, 2], [1/4, 1/3, 1/2, 1]])
    IDEAL_VALUES = {'Life Expectancy': 90, 'Education Index': 1.0, 'Gini Index': 0.25}
    for i, year in enumerate(SIMULATION_YEARS):
        threshold_factor = 2 / (1 + np.exp(0.034 * i))
        direct_national_profits = {c: 0 for c in initial_national_df['Country']}
        private_profit_pool = 0
        for idx, entity in df_all_entities.iterrows():
            amci_growth = 2 / (1 + np.exp(-0.05 * i))
            if policy_name in ['Technology Transfer', 'Combined Policies'] and initial_amci.loc[idx] < 40:
                amci_growth *= 1.5
            entity_amci = initial_amci.loc[idx] * amci_growth
            mining_class = 'None'
            if entity_amci >= ASTEROID_CLASSES['I'] * threshold_factor: mining_class = 'I'
            elif entity_amci >= ASTEROID_CLASSES['II'] * threshold_factor: mining_class = 'II'
            elif entity_amci >= ASTEROID_CLASSES['III'] * threshold_factor: mining_class = 'III'
            ramp = 1 / (1 + np.exp(-0.2 * (i - 17.5)))
            profit = {'I': 1.0E12, 'II': 0.25E12, 'III': 0.05E12, 'None': 0}[mining_class] * ramp
            if entity['Type'] == 'National': direct_national_profits[entity['Country']] += profit
            else: private_profit_pool += profit
        market_profits = private_profit_pool * (current_national_df.set_index('Country')['GDP'] / current_national_df['GDP'].sum())
        total_profits = pd.Series(direct_national_profits).add(market_profits, fill_value=0)
        if policy_name in ['Resource Sharing', 'Combined Policies']:
            fund = total_profits.sum() * 0.20
            direct_after_tax = total_profits * 0.80
            dist_weights = (1 / (current_national_df.set_index('Country')['GDP'] + 1e-10))
            final_profits = direct_after_tax.add(fund * (dist_weights / dist_weights.sum()), fill_value=0)
        else: final_profits = total_profits
        for country, profit in final_profits.items():
            idx = current_national_df.index[current_national_df['Country'] == country]
            if not idx.empty and profit > 0:
                idx, c_gdp = idx[0], current_national_df.loc[idx[0], 'GDP']
                impact_factor = np.log1p(profit / c_gdp) * 0.1
                current_national_df.loc[idx, 'GDP'] += profit
                current_national_df.loc[idx, 'Life Expectancy'] += (IDEAL_VALUES['Life Expectancy'] - current_national_df.loc[idx, 'Life Expectancy']) * impact_factor
                current_national_df.loc[idx, 'Education Index'] += (IDEAL_VALUES['Education Index'] - current_national_df.loc[idx, 'Education Index']) * impact_factor
                current_national_df.loc[idx, 'Gini Index'] -= (current_national_df.loc[idx, 'Gini Index'] - IDEAL_VALUES['Gini Index']) * impact_factor
                current_national_df.loc[idx, 'Scientific Publications'] *= (1 + impact_factor * 2)
                current_national_df.loc[idx, 'Number of Patents Filed'] *= (1 + impact_factor * 1.5)
        updated_df, _, _ = calculate_eei(current_national_df, FACTOR_GROUPS, INDICATOR_TYPES, ahp_matrix_sim)
        history.append({'Year': year, 'Mean EEI': updated_df['EEI'].mean()})
    return pd.DataFrame(history), current_national_df

# VISUALIZATION FUNCTIONS
def generate_all_visualizations(eei_df, comp_weights, pca_results, scenario_df, final_state_df, df_with_amci):
    print("\n--- Generating All Visualizations (Publication-Quality, Sized for 2-Column Journal) ---")
    print("Figures will be shown in the browser. No titles included. Sizes are approximate for 2-column (e.g., width ~3.5-7 inches at 300 DPI).")

    # Nested Pie Charts for Weights (Sunburst)
    indicator_to_factor = {ind: factor for factor, inds in FACTOR_GROUPS.items() for ind in inds}
    label_map = {'Total natural resources rents (% of GDP)': 'Nat. Res. Rents', 'R&D Expenditure (% of GDP)': 'R&D Spend', 'Scientific Publications': 'Sci. Pubs.',
                 'Number of Patents Filed': 'Patents', 'Space Agency Budget (US$ millions)': 'Space Budget', 'Most number of Objects Launched Into Space in a Year': 'Launches',
                 'Years of Space Program Experience': 'Space Exp.', 'Environmental Performance Index': 'EPI', 'Political Stability Index': 'Poli. Stability'}

    def create_sunburst_data(indicator_list, comprehensive_weights):
        sub_weights = comprehensive_weights[comprehensive_weights.index.isin(indicator_list)]
        labels = [label_map.get(i, i) for i in sub_weights.index]
        parents = [indicator_to_factor[i] for i in sub_weights.index]
        values = sub_weights.values
        factor_labels = list(set(parents))
        factor_parents = [''] * len(factor_labels)
        factor_values = [0] * len(factor_labels)
        return (labels + factor_labels, parents + factor_parents, list(values) + factor_values)

    input_labels, input_parents, input_values = create_sunburst_data(INPUT_INDICATORS, comp_weights)
    output_labels, output_parents, output_values = create_sunburst_data(OUTPUT_INDICATORS, comp_weights)

    fig_weights = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]])
    fig_weights.add_trace(go.Sunburst(labels=input_labels, parents=input_parents, values=input_values), 1, 1)
    fig_weights.add_trace(go.Sunburst(labels=output_labels, parents=output_parents, values=output_values), 1, 2)
    fig_weights.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=1000, height=500)
    fig_weights.show()
    print("Displayed Fig 1 (Hierarchical Weights)")

    # World Map
    fig_map = px.choropleth(eei_df, locations="Country", locationmode='country names', color="EEI", hover_name="Country", color_continuous_scale=px.colors.sequential.Plasma_r)
    fig_map.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=800, height=500)
    fig_map.show()
    print("Displayed Fig 2 (World Map)")

    # PCA Biplot
    pca, X_pca, indicators = pca_results['pca'], pca_results['X_pca'], pca_results['indicators']
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    fig_pca = px.scatter(X_pca, x=0, y=1, text=eei_df['Country'])
    for i, feature in enumerate(indicators):
        fig_pca.add_shape(type='line', x0=0, y0=0, x1=loadings[i, 0]*5, y1=loadings[i, 1]*5)
        fig_pca.add_annotation(x=loadings[i, 0]*5.5, y=loadings[i, 1]*5.5, text=label_map.get(feature, feature).replace(" ", "<br>"), showarrow=False)
    fig_pca.update_traces(textposition='top center')
    fig_pca.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=600, height=500, xaxis_title='Principal Component 1', yaxis_title='Principal Component 2')
    fig_pca.show()
    print("Displayed Fig 3 (PCA Biplot)")

    # AMCI Bar Chart
    df_sorted = df_with_amci.sort_values('AMCI', ascending=True)
    color_map = {'National': 'royalblue', 'Private': 'goldenrod'}
    colors = df_sorted['Type'].map(color_map)
    fig_amci = go.Figure(go.Bar(
        x=df_sorted['AMCI'],
        y=df_sorted['Agency'],
        orientation='h',
        marker_color=colors,
        text=df_sorted['AMCI'].round(2),
        textposition='outside'
    ))
    for class_name, threshold in ASTEROID_CLASSES.items():
        fig_amci.add_vline(x=threshold, line_width=2, line_dash="dash", line_color="orange")
        fig_amci.add_annotation(x=threshold, y=len(df_sorted)-0.5, text=f"Class {class_name}", showarrow=False, font=dict(size=12), xshift=25)
    fig_amci.update_layout(title_text='<b>Fig 3: Asteroid Mining Competitive Index (AMCI)</b>', height=800, xaxis_title='AMCI Score').show()


    # TOPSIS Rankings Bar Chart
    fig_bar = px.bar(eei_df.sort_values('EEI'), x='Country', y=['Input_Score', 'Output_Score'], barmode='group')
    fig_bar.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=1000, height=500,legend=dict(
            x=0.01,
            y=0.99,
            xanchor='left',
            yanchor='top'
        )
)
    fig_bar.show()
    print("Displayed Fig 4 (TOPSIS Scores Bar Chart)")

    # Scenario Comparison Line Plot
    fig_line = px.line(scenario_df, x='Year', y='Mean EEI', color='Scenario', markers=True)
    fig_line.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=600, height=400,legend=dict(
            x=0.01,
            y=0.99,
            xanchor='left',
            yanchor='top'
        )
)
    fig_line.show()
    print("Displayed Fig 5 (Scenario Comparison)")

    # Correlation Heatmap
    all_inds = list(set(INPUT_INDICATORS + OUTPUT_INDICATORS))
    corr_df = eei_df[all_inds].rename(columns=label_map)
    corr_matrix = corr_df.corr()
    fig_heatmap = px.imshow(corr_matrix, aspect="auto", color_continuous_scale='RdBu_r')
    fig_heatmap.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=600, height=500)
    fig_heatmap.show()
    print("Displayed Fig 6 (Correlation Heatmap)")

    # Tree Maps for Allocations (Side-by-side)
    color_scale = 'Viridis'
    fig_tm_current = make_subplots(rows=1, cols=2, specs=[[{'type': 'treemap'}, {'type': 'treemap'}]])
    fig_tm_current.add_trace(go.Treemap(labels=eei_df['Country'], parents=['World']*len(eei_df), values=eei_df['Input_Score'], marker_colors=eei_df['Input_Score'], marker_colorscale=color_scale), 1, 1)
    fig_tm_current.add_trace(go.Treemap(labels=eei_df['Country'], parents=['World']*len(eei_df), values=eei_df['Output_Score'], marker_colors=eei_df['Output_Score'], marker_colorscale=color_scale), 1, 2)
    fig_tm_current.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=1000, height=500)
    fig_tm_current.show()
    print("Displayed Fig 7 (Current Score Allocations)")

    fig_tm_final = make_subplots(rows=1, cols=2, specs=[[{'type': 'treemap'}, {'type': 'treemap'}]])
    fig_tm_final.add_trace(go.Treemap(labels=final_state_df['Country'], parents=['World']*len(final_state_df), values=final_state_df['Input_Score'], marker_colors=final_state_df['Input_Score'], marker_colorscale=color_scale), 1, 1)
    fig_tm_final.add_trace(go.Treemap(labels=final_state_df['Country'], parents=['World']*len(final_state_df), values=final_state_df['Output_Score'], marker_colors=final_state_df['Output_Score'], marker_colorscale=color_scale), 1, 2)
    fig_tm_final.update_layout(margin=dict(t=0, l=0, r=0, b=0), width=1000, height=500)
    fig_tm_final.show()
    print("Displayed Fig 8 (Final Score Allocations)")

def main():
    df_all = load_and_clean_data('country_indicators.csv')
    if df_all is None: return

    print("\nCalculating Current Equity & Weights")
    national_df = df_all[df_all['Type'] == 'National'].copy().reset_index(drop=True)
    ahp_matrix = np.array([[1, 2, 3, 4], [1/2, 1, 2, 3], [1/3, 1/2, 1, 2], [1/4, 1/3, 1/2, 1]])
    eei_results, comprehensive_weights, _ = calculate_eei(national_df, FACTOR_GROUPS, INDICATOR_TYPES, ahp_matrix)
    print(eei_results.sort_values('EEI')[['Country', 'Input_Score', 'Output_Score', 'EEI']].round(3))

    print("\nPerforming PCA")
    all_indicators = list(set(INPUT_INDICATORS + OUTPUT_INDICATORS))
    X = national_df[all_indicators].values
    X_scaled = StandardScaler().fit_transform(X)
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    pca_results = {'pca': pca, 'X_pca': X_pca, 'indicators': all_indicators}

    print("\nSimulating Future Impact & Policies")
    df_all_with_amci = calculate_amci(df_all.copy())
    policies = ['No Policy', 'Resource Sharing', 'Technology Transfer', 'Combined Policies']
    all_policy_results, final_state_df = [], None

    original_stdout = sys.stdout; sys.stdout = open(os.devnull, 'w')
    for policy in policies:
        sim_df, temp_final_state = run_simulation(df_all_with_amci, national_df.copy(), policy_name=policy)
        sim_df['Scenario'] = policy
        all_policy_results.append(sim_df)
        if policy == 'Combined Policies': final_state_df = temp_final_state
    sys.stdout = original_stdout

    comparison_df = pd.concat(all_policy_results).reset_index()
    print("\nAll simulations complete.")

    generate_all_visualizations(eei_results, comprehensive_weights, pca_results, comparison_df, final_state_df, df_all_with_amci)

if __name__ == '__main__':
    main()

Data loaded and cleaned successfully.

Calculating Current Equity & Weights
           Country  Input_Score  Output_Score    EEI
14            Iran        0.553         0.388  0.165
12         Algeria        0.838         0.589  0.249
1            China        2.776         2.453  0.323
3            Japan        3.432         3.875  0.444
31    South Africa        1.500         0.565  0.935
4           Russia        1.555         0.610  0.945
25       Indonesia        1.606         0.652  0.954
22          Brazil        1.649         0.684  0.965
5            India        1.682         0.557  1.125
27     Phillepines        1.731         0.582  1.148
18         Ukraine        1.534         0.327  1.207
28         Nigeria        1.360         0.116  1.245
24       Argentina        2.059         0.773  1.286
33          Mexico        2.024         0.574  1.449
23          Poland        2.807         1.124  1.683
26        Pakistan        1.730         0.030  1.701
20          Turkey     

Displayed Fig 1 (Hierarchical Weights)


Displayed Fig 2 (World Map)


Displayed Fig 3 (PCA Biplot)


Displayed Fig 4 (TOPSIS Scores Bar Chart)


Displayed Fig 5 (Scenario Comparison)


Displayed Fig 6 (Correlation Heatmap)


Displayed Fig 7 (Current Score Allocations)


Displayed Fig 8 (Final Score Allocations)
